# Setup LLM

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables.base import Runnable
import os

def chat_openai() -> Runnable:
    openai_api_key = os.getenv("OPENAI_API_KEY")
    llm = ChatOpenAI(
        openai_api_key=openai_api_key,
        model = "gpt-4o-mini",
        temperature=0
        )
    return llm

llm = chat_openai()
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x112e04f90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x113683010>, root_client=<openai.OpenAI object at 0x111bf7150>, root_async_client=<openai.AsyncOpenAI object at 0x112ee3f50>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

# Input

In [ ]:
# Global variable
input = """
Rescuers are rushing to find people still trapped under collapsed buildings in quake-hit Myanmar as the crucial 72-hour mark approaches, when survival rates are said to drop sharply.
Friday's magnitude 7.7 earthquake has claimed about 1,700 lives and injured some 3,400 people. Around 300 others remain unaccounted for.
Mandalay, the second-largest city, saw widespread damage, including many collapsed buildings. The search continues for those believed to be trapped inside a large condominium building that mostly collapsed.
Local officials say a Japanese national who lived in the building is unaccounted for. Japan's Foreign Ministry has asked Myanmar authorities, through its embassy in the country, to search for and rescue the Japanese national.
Damage has also been reported in neighboring Thailand. A high-rise building under construction collapsed in the capital Bangkok.
Thai authorities say at least 11 people were killed and more than 70 others remain unaccounted for.
Search and rescue operations continue using large cranes and other equipment. More rescue workers are being brought in to speed up the search." \
"""

SyntaxError: incomplete input (3887204207.py, line 2)

# 1) Question 1 (Summarization)

## 1.1) Prompt

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt_summarization = ChatPromptTemplate.from_template("""
Prompt...

Here is the text:
{input}
"""
)

prompt_summarization.messages

## 1.2) Chain

In [ ]:
# Define chain summarization
chain_summarization = prompt_summarization | llm
chain_summarization

## 1.3) Answer

In [ ]:
response = chain_summarization.invoke({
    "input": input
})
response.content

# 2) Question 2 (Pydantic Parser)
##### The answer should include:
##### 1) The affected countries
##### 2) The magnitude of the earthquake.
##### 3) The number of injured people.

## 2.1) Prompt

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt_extraction = ChatPromptTemplate.from_template("""
Prompt...

Here is the text:
{input}
"""
)

prompt_extraction.messages

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from typing import List, Dict, Any
from pydantic import BaseModel, Field

class Output(BaseModel):
    location: str = Field(description="...")
    # ...

parser = PydanticOutputParser(pydantic_object=Output)
print(parser.get_format_instructions())

In [ ]:
from langchain_core.messages import HumanMessage

# Adding context of the output parser to the prompt
prompt_extraction.messages.append(HumanMessage(
    content=parser.get_format_instructions()
))
prompt_extraction.messages

## 2.2) Chain

In [ ]:
# Define chain extraction
chain_extraction: Runnable 

## 2.3) Answer

In [ ]:
response = chain_extraction.invoke({
    "input": input
})
response

# 3) Question 3 (Translation Chain & Consolidate to Sequential Chain) [OPTIONAL]
##### Summarize -> Translation
##### The answer should be provided by 1 invoke execution

## 3.1) Prompt

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt_translation = ChatPromptTemplate.from_template("""
Prompt...

Here is the text:
{input}
"""
)

prompt_extraction.messages

## 3.2) Chain

In [ ]:
# Use chain summaize from question 1
chain_summarize = prompt_extraction | llm
chain_summarize

In [ ]:
# Define chain translation
chain_translation: Runnable

In [ ]:
# Consolidate to Sequencial Chain
chain_final: Runnable

## 3.3) Answer

In [ ]:
response = chain_final.invoke({
    "input": input
})